# 機械特性を計算するための準備

## Quantum espressoでSiO2の機械特性を計算する

In [3]:
from mlptools.io.write import QuantumEspressoWriter
from ase.io.vasp import read_vasp

In [4]:
path2atom = "/Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-559091/original/POSCAR"
atoms = read_vasp(path2atom)

In [6]:
atoms.cell[:]

array([[ 2.53453694, -4.38994676,  0.        ],
       [ 2.53453694,  4.38994676,  0.        ],
       [ 0.        ,  0.        ,  8.55848854]])

In [15]:
for symbol, scaled_position in zip(atoms.get_chemical_symbols(), atoms.get_scaled_positions()):
    print(" ".join([symbol] + list(map(str, scaled_position))))

Si 0.6666666666666666 0.3333333333333333 0.9371258
Si 0.6666666666666666 0.33333333333333337 0.5628742
Si 0.3333333333333335 0.6666666666666667 0.0628742
Si 0.33333333333333337 0.6666666666666666 0.4371258
O 0.0 0.4212587400000001 0.0
O 0.4212587400000001 0.4212587400000001 0.5
O 0.5787412599999999 0.0 0.5
O 0.5787412599999999 0.5787412599999999 0.0
O 0.0 0.5787412599999999 0.5
O 0.6666666666666666 0.3333333333333333 0.7500000000000001
O 0.33333333333333337 0.6666666666666666 0.25
O 0.4212587400000001 0.0 0.0


# LAMMPS用に結晶データ作成する

In [18]:
from ase.io.lammpsdata import write_lammps_data

path2atom = "/Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-546794/original/POSCAR"
atoms = read_vasp(path2atom)

In [ ]:
with open("/Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-546794/elastic/lmp/atom_skew.data", "w") as f:
    write_lammps_data(f, atoms, force_skew=True)